# Convolutional Neural Network (CNN) Demonstration

## Import packages

In [1]:
! pip install torch torchvision pandas tqdm pathlib

  Using cached pathlib-1.0.1-py3-none-any.whl.metadata (5.1 kB)
Using cached pathlib-1.0.1-py3-none-any.whl (14 kB)


In [2]:
import torch
import torch.nn as nn

import torchvision as tv
from torchvision import datasets
from torch.utils.data import DataLoader

import pandas as pd
from tqdm import tqdm
from pathlib import Path
from functools import reduce

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Models Implementation

### Feed-Forward Model Implementation

In [3]:
class FeedForwardClassifier(nn.Module):
  def __init__(self, input_shape, n_classes, hidden_units=1024, dropout_rate=0.25):
    super(FeedForwardClassifier, self).__init__()

    input_dim = reduce(lambda x, y: x * y, input_shape)

    self.classifier = nn.Sequential(
      nn.Flatten(),

      nn.Linear(input_dim, hidden_units),
      nn.ReLU(),
      nn.Dropout(dropout_rate),

      nn.Linear(hidden_units, hidden_units),
      nn.ReLU(),
      nn.Dropout(dropout_rate),

      nn.Linear(hidden_units, n_classes),
    )

  def forward(self, x):
    return self.classifier(x)

### CNN Model Implementation

In [4]:
class CNNClassifier(nn.Module):
  def __init__(self, input_shape, n_classes, cnn_hidden_units=32, linear_hidden_units=1024, dropout_rate=0.25):
    super(CNNClassifier, self).__init__()

    self.conv_block1 = nn.Sequential(
      nn.Conv2d(input_shape[0], cnn_hidden_units, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(cnn_hidden_units, cnn_hidden_units, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Dropout(dropout_rate),
    )
    self.conv_block2 = nn.Sequential(
      nn.Conv2d(cnn_hidden_units, cnn_hidden_units, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(cnn_hidden_units, cnn_hidden_units, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2),
      nn.Dropout(dropout_rate),
    )
    self.classifier = nn.Sequential(
      nn.Flatten(),
      nn.Linear(cnn_hidden_units * 7 * 7, linear_hidden_units),
      nn.Dropout(dropout_rate),
      nn.Sigmoid(),
      nn.Linear(linear_hidden_units, n_classes),
    )

  def forward(self, x):
    x = self.conv_block1(x)
    x = self.conv_block2(x)

    x = x.view(x.size(0), -1)

    x = self.classifier(x)

    return x

## Perform Experiment

### Define Accuracy Function

In [5]:
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct / len(y_pred)) * 100
  return acc

### Setup PyTorch device

In [6]:
device = (
  'cuda'
  if torch.cuda.is_available()
  else 'mps'
  if torch.backends.mps.is_available()
  else 'cpu'
)

### Define Function for Model Evaluation

In [7]:
def eval_model(model, data_loader, loss_fn, accuracy_fn):
  loss, acc = 0, 0

  model.eval()

  with torch.inference_mode():
    for x, y in data_loader:
      x, y = x.to(device), y.to(device)

      y_pred = model(x)

      loss += loss_fn(y_pred, y)
      acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))

    loss /= len(data_loader)
    acc /= len(data_loader)

  return {'name': model.__class__.__name__, 'loss': loss.item(), 'acc': acc}

### Define Function for Model Evaluation

In [8]:
def get_n_params(model):
  total_params = 0

  for param in list(model.parameters()):
    num_elements = 1

    for size in list(param.size()):
      num_elements *= size

    total_params += num_elements

  return total_params

### Define Experiment Grid

In [9]:
example_datasets = (datasets.EMNIST, datasets.FashionMNIST, datasets.CIFAR100)
models = (FeedForwardClassifier, CNNClassifier)

### Perform Training and Evaluation

In [10]:
data_dir = Path('data')

batch_size = 32
epochs = 10

torch.manual_seed(42)

results_df = pd.DataFrame()

for Dataset in tqdm(example_datasets, desc='Datasets'):
  train_data = datasets.FashionMNIST(
    root=data_dir / Dataset.__name__,
    train=True,
    download=True,
    transform=tv.transforms.ToTensor(),
    target_transform=None,
  )
  test_data = datasets.FashionMNIST(
    root=data_dir / Dataset.__name__,
    train=False,
    download=True,
    transform=tv.transforms.ToTensor(),
  )

  image, label = train_data[0]
  input_shape = image.shape
  class_names = train_data.classes

  train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=batch_size,
    shuffle=True,
  )
  test_dataloader = DataLoader(
    dataset=test_data,
    batch_size=batch_size,
    shuffle=False,
  )

  for Model in tqdm(models, desc='Models'):
    model = Model(input_shape=image.shape, n_classes=len(class_names)).to(device)
    n_params = get_n_params(model)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

    for epoch in tqdm(range(epochs), desc='Epochs'):
      train_loss = 0

      for batch, (x, y) in enumerate(train_dataloader):
        x, y = x.to(device), y.to(device)

        model.train()

        y_pred = model(x)

        loss = loss_fn(y_pred, y)
        train_loss += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

      train_loss /= len(train_dataloader)

      test_loss, test_acc = 0, 0

      model.eval()

      with torch.inference_mode():
        for x, y in test_dataloader:
          x, y = x.to(device), y.to(device)

          test_pred = model(x)

          test_loss += loss_fn(test_pred, y)

          test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))

          test_loss /= len(test_dataloader)
          test_acc /= len(test_dataloader)

    results = eval_model(model, test_dataloader, loss_fn, accuracy_fn)
    row = pd.DataFrame({
      'dataset': Dataset.__name__,
      'model': Model.__name__,
      'n_params': n_params,
      'loss': results['loss'],
      'acc': results['acc'],
    }, index=[0])
    results_df = pd.concat([results_df, row], ignore_index=True)

results_df

Datasets:   0%|          | 0/3 [00:00<?, ?it/s]












Epochs: 100%|██████████| 10/10 [01:06<00:00,  6.66s/it]











Datasets:  33%|███▎      | 1/3 [02:45<05:31, 165.65s/it]










Epochs: 100%|██████████| 10/10 [01:02<00:00,  6.26s/it]











Datasets:  67%|██████▋   | 2/3 [05:23<02:41, 161.33s/it]










Epochs: 100%|██████████| 10/10 [01:04<00:00,  6.49s/it]











Datasets: 100%|██████████| 3/3 [08:08<00:00, 162.85s/it]


,dataset,model,n_params,loss,acc
0,EMNIST,FeedForwardClassifier,1863690,0.317993,88.278754
1,EMNIST,CNNClassifier,1644970,0.251149,91.902955
2,FashionMNIST,FeedForwardClassifier,1863690,0.343027,87.699681
3,FashionMNIST,CNNClassifier,1644970,0.274849,90.674920
4,CIFAR100,FeedForwardClassifier,1863690,0.322914,88.428514
5,CIFAR100,CNNClassifier,1644970,0.292897,90.684904
